## 1. 텍스트 생성하기

In [2]:
# pip install langchain
# pip install langchain-openai
# pip install python-dotenv

from dotenv import load_dotenv
load_dotenv()
import openai
import os
openai.api_key= os.environ.get("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

llm.invoke("너는 어떤 기능을 할 수 있어?")

AIMessage(content='저는 대화를 주고받을 수 있고, 질문에 답변을 제공하며 정보를 제공할 수 있습니다. 또한 일부 작업을 자동화하거나 일정을 관리하는 등의 기능도 수행할 수 있습니다. 필요한 경우에는 기능을 추가하여 다양한 작업을 수행할 수 있습니다.', response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 23, 'total_tokens': 123}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-082f9700-0cde-4718-85b1-fac49218f30f-0')

## 2. 프롬프트 작성하기

In [3]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 여행 블로그를 작성하는 에이전트야"),
    ("user", "{input}")
])

chain = prompt | llm 

chain.invoke({"input": "파리"})

AIMessage(content='파리, 프랑스의 수도로서 아름다운 도시로 유명합니다. 파리는 세계적으로 유명한 랑브르 박물관, 에펠탑, 개선문, 성 드 노트르담 대성당 등 많은 명소를 보유하고 있습니다. 또한 파리는 로맨틱한 분위기와 유럽의 역사와 문화를 만끽할 수 있는 도시로서 많은 여행객들이 방문하는 인기 여행지입니다.\n\n파리를 방문할 때는 반드시 에펠탑을 오르는 것을 권해드립니다. 에펠탑은 파리의 상징이자 대표적인 명소로서 야경이 아름다운 곳으로 유명합니다. 또한 루브르 박물관은 세계적으로 유명한 미술작품들을 감상할 수 있는 곳으로, 예술을 즐기는 여행객들에게 추천합니다.\n\n파리에는 유명한 베이커리와 카페들이 많아서 프랑스의 다양한 디저트와 커피를 즐길 수 있습니다. 또한 샹젤리제 대로는 쇼핑을 즐기기에 좋은 장소이며, 몽마르트르 지역은 아티스트들이 모여 있는 예술가의 마을로 유명합니다.\n\n파리는 역사와 문화, 먹거리 등 다양한 즐길 거리가 있는 도시로 여행을 계획하신다면 꼭 방문해보셨으면 좋겠습니다. 파리의 아름다움과 매력에 푹 빠지실 것입니다.', response_metadata={'token_usage': {'completion_tokens': 528, 'prompt_tokens': 33, 'total_tokens': 561}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-39874211-ef86-482d-9b70-e6c1da8b5823-0')

## 3. 답변만 따로 받기

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"input": "파리"})

'파리는 아름다운 도시로 유명한 프랑스의 수도입니다. 이곳에는 에펠 탑, 루브르 박물관, 샹젤리제 거리, 장 드 마르슈 시장 등 많은 관광 명소가 있습니다. 또한 파리는 유럽의 문화 중심지로서 예술, 음악, 모던 아트 등 다양한 분야에서 활발한 활동이 이루어지는 도시입니다. 파리의 골목길을 거닐며 맛있는 프랑스 음식을 맛보고, 세계적인 작품들을 감상할 수 있는 여행은 정말 특별한 경험이 될 것입니다. 파리를 여행하면서 느끼는 감정과 추억을 담아 블로그를 작성해보는 것은 어떨까요?'

## 4. 챗봇 만들기 

In [5]:
# pip install beautifulsoup4
# pip install faiss-cpu

# 1) 데이터 로딩하기
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://ko.wikipedia.org/wiki/%EC%82%AC%EA%B3%BC_%EC%A3%BC%EC%8A%A4")

docs = loader.load()

# 2) 텍스트 토큰화하기
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

# 3) 데이터 쪼개기
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

# 4) 데이터베이스에 업로드하기
vector = FAISS.from_documents(documents, embeddings)

# 5) 데이터에 대해 질문과 답변할 수 있는 프롬프트 및 모델 만들기
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

# 6) 업로드한 데이터를 불러올 수 있는 retriever 체인 만들기
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# 7) 답변받기
response = retrieval_chain.invoke({"input": "사과주스는 건겅에 어떻게 좋은가요?"})
print(response["answer"])

사과 주스에는 건강한 뼈를 촉진시킬 수 있는 붕소를 포함한 다양한 미네랄 영양분이 함유되어 있어 건강에 도움이 될 수 있습니다.


## 5. 에이전트 만들기 (스스로 다음 단계를 생각하면서 답변하는 에이전트)

In [6]:
# pip install langchainhub
# pip install langchain-openai

# 에이전트가 활용할 수 있는 툴을 만들어줌
from langchain.tools.retriever import create_retriever_tool

# 1. 검색 툴
retriever_tool = create_retriever_tool(
    retriever,
    "apple_search",
    "Search for information about Apple Juice. For any questions about LangSmith, you must use this tool!",
)

# 2. 실시간 검색 툴
# https://tavily.com/ 회원가입 후 API KEY 받기
# 터미널에 export TAVILY_API_KEY=...
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

tools = [retriever_tool, search]

from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# You need to set OPENAI_API_KEY environment variable or pass it as argument `api_key`.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
agent_executor.invoke({"input": "사과주스는 건겅에 어떻게 좋은가요?"})



> Entering new AgentExecutor chain...

Invoking: `apple_search` with `{'query': 'health benefits of apple juice'}`


사과 주스 - 위키백과, 우리 모두의 백과사전





































본문으로 이동







주 메뉴





주 메뉴
사이드바로 이동
숨기기



		둘러보기
	


대문최근 바뀜요즘 화제임의의 문서로기부





		사용자 모임
	


사랑방사용자 모임관리 요청





		편집 안내
	


소개도움말정책과 지침질문방



















검색











검색





























계정 만들기

로그인








개인 도구





 계정 만들기 로그인





		로그아웃한 편집자를 위한 문서 더 알아보기



기여토론




























목차
사이드바로 이동
숨기기




처음 위치





1건강에 미치는 영향







2저온 살균







3각주

















목차 토글







사과 주스



39개 언어




العربيةБеларускаяTsetsêhestâheseČeštinaDanskDeutschΕλληνικάEnglishEsperantoEspañolEestiفارسیSuomiFrançaisעבריתՀայերենBahasa IndonesiaItaliano日本語JawaLietuviųNederlandsNorsk nynorskNorsk bokmålPolskiPortuguêsРусскийसंस्कृतम्Simple EnglishSlovenščinaShqipСрпски / srpskiSvenskaTagalogTürkçeУкраїнськаTiếng Việt中文粵語

링크 편집











문서토론





한국어

















읽기편집역사 보기







도구


{'input': '사과주스는 건겅에 어떻게 좋은가요?',
 'output': '사과 주스는 건강에 다양한 영향을 줄 수 있습니다. 사과 주스에는 건강한 뼈를 촉진시킬 수 있는 붕소를 비롯한 다양한 미네랄 영양분이 함유되어 있습니다. 또한, 사과 주스는 비타민 C가 종종 추가되어 있어 영양 강화에 도움을 줄 수 있습니다. 하지만 주스의 내용물이 가변적이고 가공 중 일부 영양소가 소실될 수 있으니 적당한 섭취량을 유지하는 것이 중요합니다.'}

In [8]:
agent_executor.invoke({"input": "한국의 날씨가 어떤가요?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '한국 날씨'}`


[{'url': 'https://weather.naver.com/', 'content': "We would like to show you a description here but the site won't allow us."}, {'url': 'https://www.accuweather.com/ko/kr/south-korea-weather', 'content': '버지니아. 36° F. RealFeel 37°. 대한민국 기상 레이더 더 보기. 대한민국 기상 상태 더 보기. 고양시 53° 광주광역시 55° 기흥구 55° 단원구 55° 대구 ...'}, {'url': 'https://www.weather.go.kr/weather/home/index.jsp', 'content': '기상청 날씨누리는 전국의 1시간 관측 날씨, 기온, 강수, 풍향, 풍속 등을 제공하고, 내일은 전국 대부분에 비가 오고 바람이 강하게 부는 곳이 있습니다. 또한'}, {'url': 'https://www.weather.go.kr/weather/main.jsp', 'content': '기상청 날씨누리. 전국특보 발효중. 밀물·썰물 정보 해와 달이 뜨고지는 시간 기후정보포털 기상자료 개방포털 기상기후 빅데이터 대기질 예·경보 날씨해설 (유튜브) 기상청 행정누리집 산악날씨 한강홍수통제소 해양기상 정보포털 항공기상청.'}, {'url': 'https://web.kma.go.kr/eng/weather/forecast/current_korea.jsp', 'content': 'Station Weather Visibility (km) Cloud (1/10) Temp. (℃) Wind Dir. Wind Speed (m/s) Humidity (%) Daily Precipitation (mm) Air Pressure 

{'input': '한국의 날씨가 어떤가요?',
 'output': '한국의 날씨 정보를 확인하려면 [기상청 날씨누리](https://www.weather.go.kr/weather/home/index.jsp) 웹사이트를 방문하시면 됩니다. 현재 전국 대부분에 비가 오고 바람이 강하게 부는 곳이 있습니다. 추가적인 상세한 정보를 원하시면 해당 웹사이트를 참고해 주세요.'}